<a href="https://colab.research.google.com/github/Menna1410/File-Organizer/blob/main/file_organizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [1]:
# For File Organizer we don't need extra packages
# Just confirm Python version
!python --version

Python 3.11.13


# Main Code

In [2]:
import os
import shutil
from datetime import datetime
from typing import Dict

# File extension → category mapping
EXT_MAP: Dict[str, str] = {
    ".jpg": "Images", ".jpeg": "Images", ".png": "Images", ".gif": "Images",
    ".bmp": "Images", ".tiff": "Images", ".svg": "Images", ".webp": "Images",
    ".pdf": "Documents", ".doc": "Documents", ".docx": "Documents",
    ".txt": "Documents", ".rtf": "Documents", ".odt": "Documents",
    ".xls": "Documents", ".xlsx": "Documents", ".ppt": "Documents", ".pptx": "Documents",
    ".md": "Documents",
    ".mp4": "Videos", ".mkv": "Videos", ".mov": "Videos", ".avi": "Videos",
    ".wmv": "Videos", ".flv": "Videos", ".webm": "Videos",
    ".mp3": "Others", ".wav": "Others", ".aac": "Others",
}

def ensure_dir(path: str):
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)

def categorize_filename(filename: str) -> str:
    _, ext = os.path.splitext(filename.lower())
    return EXT_MAP.get(ext, "Others")

def organize_folder(folder: str, simulate: bool=False) -> Dict[str, int]:
    if not os.path.isdir(folder):
        raise FileNotFoundError(f"Folder not found: {folder}")
    moved_counts: Dict[str, int] = {}
    for name in os.listdir(folder):
        full_path = os.path.join(folder, name)
        if os.path.isfile(full_path):
            category = categorize_filename(name)
            dest_dir = os.path.join(folder, category)
            dest_path = os.path.join(dest_dir, name)
            ensure_dir(dest_dir)
            if simulate:
                print(f"[SIMULATE] Would move: {name} → {category}/")
            else:
                if os.path.exists(dest_path):
                    base, ext = os.path.splitext(name)
                    stamp = datetime.now().strftime("%Y%m%d%H%M%S")
                    dest_path = os.path.join(dest_dir, f"{base}_{stamp}{ext}")
                shutil.move(full_path, dest_path)
                print(f"Moved: {name} → {category}/")
            moved_counts[category] = moved_counts.get(category, 0) + 1
    return moved_counts

#Testing

In [4]:
# Create test folder and dummy files
test_folder = "test_files"
ensure_dir(test_folder)

sample_files = ["photo.jpg", "doc.pdf", "video.mp4", "notes.txt", "audio.mp3"]
for fname in sample_files:
    with open(os.path.join(test_folder, fname), "w") as f:
        f.write("dummy content")

# Run in simulate mode
print("=== Simulate ===")
organize_folder(test_folder, simulate=True)

# Run actual move
print("\n=== Organize ===")
result = organize_folder(test_folder, simulate=False)

print("\nSummary:")
for cat, count in result.items():
    print(f"{cat}: {count}")
print(f"Total files moved: {sum(result.values())}")

=== Simulate ===
[SIMULATE] Would move: doc.pdf → Documents/
[SIMULATE] Would move: notes.txt → Documents/
[SIMULATE] Would move: video.mp4 → Videos/
[SIMULATE] Would move: audio.mp3 → Others/
[SIMULATE] Would move: photo.jpg → Images/

=== Organize ===
Moved: doc.pdf → Documents/
Moved: notes.txt → Documents/
Moved: video.mp4 → Videos/
Moved: audio.mp3 → Others/
Moved: photo.jpg → Images/

Summary:
Documents: 2
Videos: 1
Others: 1
Images: 1
Total files moved: 5
